# Data 620 - Week3 Group Project #1

# Team Members
* # Ahm Hamza, Ali Ahmed, Nilsa Bermudez, Ursula Podosenin


# Overview

In [49]:
import pandas as pd
import numpy as np
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx


In [50]:
members = pd.read_csv("https://raw.githubusercontent.com/nilsabermudez/Data620/main/Week3/HS116_members.csv")
votes = pd.read_csv("https://raw.githubusercontent.com/nilsabermudez/Data620/main/Week3/HS116_votes.csv")

votes_small = votes.head(1000)

top_icpsr = votes_small['icpsr'].unique()

members_small = members[members['icpsr'].isin(top_icpsr)]

In [51]:
# First look at the data
print(members_small.keys())
#print(votes_small.head())

print(members_small['party_code'])

Index(['congress', 'chamber', 'icpsr', 'state_icpsr', 'district_code',
       'state_abbrev', 'party_code', 'occupancy', 'last_means', 'bioname',
       'bioguide_id', 'born', 'died', 'nominate_dim1', 'nominate_dim2',
       'nominate_log_likelihood', 'nominate_geo_mean_probability',
       'nominate_number_of_votes', 'nominate_number_of_errors', 'conditional',
       'nokken_poole_dim1', 'nokken_poole_dim2'],
      dtype='object')
0      200
1      100
2      200
3      200
4      200
      ... 
445    200
446    200
447    200
449    100
450    200
Name: party_code, Length: 434, dtype: int64


# Convert Votes from Text to Number

In [52]:
vote_counts = votes_small.notna().sum(axis=1)

top_voters = vote_counts.sort_values(ascending=False).head(500).index

vote_matrix_small = votes_small.loc[top_voters]

vote_map = {
    'Yea': 1, 'Aye': 1, 'Yes': 1,
    'Nay': -1, 'No': -1,
    'Present': 0, 'Not Voting': 0, 'Abstain': 0
}
vote_matrix = votes_small.set_index('icpsr').applymap(lambda x: vote_map.get(x, 0))

C:\Users\oracl\AppData\Local\Temp\ipykernel_36476\2862028671.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  vote_matrix = votes_small.set_index('icpsr').applymap(lambda x: vote_map.get(x, 0))


# Compute Cosine Similarity Between Members

In [53]:
similarity = cosine_similarity(vote_matrix)

sim_matrix = cosine_similarity(vote_matrix)
sim_df = pd.DataFrame(sim_matrix, index=vote_matrix.index, columns=vote_matrix.index)
sim_df.index.name = None
sim_df.columns.name = None



# Create Edge Lst 

In [54]:
edges = sim_df.stack().reset_index()
edges.columns = ['source', 'target', 'weight']
edges = edges[edges['source'] < edges['target']]
edges = edges[edges['weight'] > 0.3]

#  Build the Network Graph

In [55]:
G = nx.Graph()

party_map = {
    100: 'Democrat',
    200: 'Republican',
    328: 'Independent'
}

members['current_party'] = members['party_code'].map(party_map).fillna('Other')
party_dict = members.set_index('icpsr')['current_party'].to_dict()

for node in vote_matrix.index:
    G.add_node(node, party=party_dict.get(node, 'Unknown'))

G.add_weighted_edges_from(edges[['source', 'target', 'weight']].itertuples(index=False))

print(f"Graph has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")


Graph has 434 nodes and 0 edges


# Calculate Centrality Measures

In [56]:
# Compute centralities
deg_cent = nx.degree_centrality(G)
eig_cent = nx.eigenvector_centrality(G, weight='weight', max_iter=3000)

# Create DataFrame using icpsr as key
centrality_df = pd.DataFrame({
    'icpsr': list(deg_cent.keys()),
    'degree_centrality': list(deg_cent.values()),
    'eigenvector_centrality': list(eig_cent.values())
})

# Merge with members to get party and name info
centrality_df = centrality_df.merge(
    members[['icpsr', 'bioname', 'current_party']],
    on='icpsr',
    how='left'
)

# Optional: rename bioname to name for clarity
centrality_df = centrality_df.rename(columns={'bioname': 'name'})


# Summary Statistics by Party

In [57]:
summary_stats = centrality_df.groupby('current_party')[['degree_centrality', 'eigenvector_centrality']].describe()
print(summary_stats)


              degree_centrality                                     \
                          count mean  std  min  25%  50%  75%  max   
current_party                                                        
Democrat                  235.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
Republican                199.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

              eigenvector_centrality                                    \
                               count      mean           std       min   
current_party                                                            
Democrat                       235.0  0.048002  6.953705e-18  0.048002   
Republican                     199.0  0.048002  6.956394e-18  0.048002   

                                                       
                    25%       50%       75%       max  
current_party                                          
Democrat       0.048002  0.048002  0.048002  0.048002  
Republican     0.048002  0.048002  0.048002  0.048002 